In [1]:
import random
import numpy as np
import pandas as pd
import math, random
from scipy import special
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime, timedelta

In [2]:
def generate_time_of_event(n=100, _lambda=5):
  arrival_time = 0
  event_times = []

  for i in range(n):
    p = random.random()
    inter_arrival_time = -math.log(1.0 - p) / _lambda

    arrival_time += inter_arrival_time
    event_times.append(arrival_time)
  
  return event_times

In [3]:
def load_df_prowgen():
  df_prowgen = pd.read_csv("datasets/test-requests3.dat", 
                   sep="\s+", 
                   skiprows=1, 
                   names=['id','file_size'])

  print("Tamanho total do dataset:", len(df_prowgen))
  
  events_time = generate_time_of_event(len(df_prowgen), 1.5)
  df_prowgen['request_time'] = events_time
  df_prowgen = df_prowgen.rename({
    'id': 'object_ID',
    'file_size': 'size'
  }, axis=1)
  
  
  
  return df_prowgen[['object_ID', 'request_time', 'size']]

In [4]:
df = load_df_prowgen()

Tamanho total do dataset: 483967


In [5]:
def load_synt():
  df = pd.read_csv("datasets/syntheticDataset_O50.csv")

  random.seed(10)
  res = [round(random.uniform(5,100) * 1000) for i in range(50)]

  id_size = {}
  for i, size in enumerate(res):
    id_size[i + 1] = size
    
  df['size'] = df['object_ID'].apply(lambda _id: id_size[_id])
  
  return df

In [27]:
df = load_synt()

In [7]:
display(df.head())

display(df.tail())

,object_ID,request_time,size
0,3,0.972871,59919
1,1,1.207572,59283
2,1,1.496262,59283
3,20,2.023976,48399
4,29,2.777488,5385


,object_ID,request_time,size
287658,42,304950.049238,26308
287659,42,305015.143156,26308
287660,42,305060.330213,26308
287661,42,305065.952156,26308
287662,42,305176.878168,26308


In [8]:
min_request_time = df['request_time'].min()

df['current_day'] = (df['request_time'] - min_request_time) / 86400
df["current_day"] = df["current_day"].apply(int)
df["current_day"] = df["current_day"] + 1

In [9]:
START_DATETIME = datetime(year=2018, month=8, day=1, hour=0, minute=0, second=0)

df['time'] = df['request_time'].apply(lambda r: START_DATETIME + timedelta(seconds=r))
df = df.set_index('time')

In [10]:
BIN_SECONDS = 3600

df.sort_values(by=['request_time'], inplace=True)
bins = np.arange(np.ceil(df.request_time.min()), np.ceil(df.request_time.max()), BIN_SECONDS)
df['hour'] = pd.cut(df['request_time'], bins, labels=np.arange(0, len(bins)-1))
df.dropna(inplace=True)

In [11]:
#df['hour'].value_counts().rename_axis('hour').reset_index(name='total').sort_values("hour").plot.bar("hour", "total", figsize=(20, 8))

In [12]:
df['x'] = 1
df = df.groupby(['object_ID', 'hour', 'size']).rolling('1H')['x'].count().reset_index().sort_values('time').set_index('time')
df.rename({'x': 'total_requests_1H'}, axis=1, inplace=True)

In [13]:
tmp_df = df.groupby(['object_ID', 'hour']).rolling('2H')['total_requests_1H'].mean().reset_index().sort_values('time').set_index('time')
df['mean_requests_2H'] = tmp_df['total_requests_1H']

In [14]:
tmp_df = df.copy()
tmp_df['x'] = 1
tmp_df = tmp_df.groupby(['object_ID']).rolling('30D')['x'].sum().reset_index().sort_values('time').set_index('time')

df['count'] = tmp_df['x']

In [15]:
def items_to_keep_in_cache(df, window=15):
  keep_in_cache = []
  for i, r in df.reset_index().iterrows():
    current_id = r['object_ID']
    rolling_df = df.iloc[i + 1:i+window]['object_ID'].values

    keep_in_cache.append(current_id in rolling_df)
  
  return keep_in_cache

In [16]:
df['cache'] = items_to_keep_in_cache(df, 15)

In [17]:
df.to_csv("final_datasets/teste.csv")